# 1. Install and Import Dependencies

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 136.9 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [13]:
comment = "Here's why I think chatGPT still can't take over your job yet, your company needs to be willing to feed in the entire existing codebase to chatgpt. Otherwise, how can chatgpt make modifications to the existing code?"

In [14]:
tokens = tokenizer.encode(comment, return_tensors='pt')

In [15]:
result = model(tokens)

In [16]:
result.logits

tensor([[ 0.1443,  0.3198,  0.4527, -0.0326, -0.8486]],
       grad_fn=<AddmmBackward0>)

In [17]:
int(torch.argmax(result.logits))+1

3

# 4. Collect Reviews

In [27]:
soup.findAll('p')

[<p class="header-link-text__09f24__LhRgF css-gutk1c" data-font-weight="semibold">Write a Review</p>,
 <p class="css-nyjpex" data-font-weight="bold">Flat White</p>,
 <p class="css-nyjpex" data-font-weight="bold">Acai Smoothie</p>,
 <p class="css-nyjpex" data-font-weight="bold">Iced Coffee</p>,
 <p class="css-nyjpex" data-font-weight="bold">Chocolate Hazelnut Smoothie</p>,
 <p class="css-nyjpex" data-font-weight="bold">Apple Fresh Juice</p>,
 <p class="css-nyjpex" data-font-weight="bold">Iced Latte</p>,
 <p class="css-nyjpex" data-font-weight="bold">Iced Tea</p>,
 <p class="css-war30n" data-font-weight="semibold"></p>,
 <p class="css-r9996t" data-font-weight="semibold"><a class="css-1idmmu3" href="/map/social-brew-cafe-pyrmont" role="link"><span class="raw__09f24__T4Ezm">224 Harris St</span></a></p>,
 <p class="css-1sb02f4" data-font-weight="bold"><span class="raw__09f24__T4Ezm">Pyrmont New South Wales 2009</span></p>,
 <p class="css-1sb02f4" data-font-weight="bold"><span class="raw__09

In [18]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [19]:
reviews

["Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent.",
 "Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent.",
 'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the

# 5. Load Reviews into DataFrame and Score

In [28]:
import numpy as np
import pandas as pd

In [29]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [35]:
df['review'].iloc[1]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent."

In [36]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [37]:
sentiment_score(df['review'].iloc[1])

4

In [41]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [39]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,Great food amazing coffee and tea. Short walk ...,5
3,Great staff and food. Must try is the pan fri...,5
4,Ricotta hot cakes! These were so yummy. I ate ...,5
5,I came to Social brew cafe for brunch while ex...,5
6,We came for brunch twice in our week-long visi...,4
7,It was ok. The coffee wasn't the best but it w...,3
8,I went here a little while ago- a beautiful mo...,2
9,This place is a gem. The ambiance is to die fo...,3


In [42]:
df['review'].iloc[3]

'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'